In [1]:
from policy.skill.config import get_model
from policy.dataset.ms2dataset import get_MS_loaders
from policy.config import load_config

In [2]:
import torch

cfg_path = "/home/mrl/Documents/Projects/tskill/assets/skill/default.yaml"
cfg = load_config(cfg_path)
model = get_model(cfg)
nparameters = sum(p.numel() for p in model.parameters())
n_trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of trainable parameters: %.2fM" % (n_trainable_parameters/1e6,))
print("Number of total parameters: %.2fM" % (nparameters/1e6,))

/home/mrl/anaconda3/envs/tskill/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mrl/anaconda3/envs/tskill/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/mrl/anaconda3/envs/tskill/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but

Number of trainable parameters: 125.15M
Number of total parameters: 125.15M


In [3]:
train_loader, val_loader = get_MS_loaders(cfg)

In [4]:
# from policy.skill.training import Trainer

# param_dicts = [{"params": [p for n, p in model.named_parameters() if "backbone" not in n and p.requires_grad]}]
# if cfg["training"]["lr_state_encoder"] > 0:
#     param_dicts.append({
#         "params": [p for n, p in model.named_parameters() if "backbone" in n and p.requires_grad],
#         "lr": cfg["training"]["lr_state_encoder"],
#     })
# lr = cfg["training"].get("lr", 1e-3)
# weight_decay = cfg["training"].get("weight_decay", 1e-4)
# optimizer = torch.optim.AdamW(param_dicts, lr=lr,
#                                   weight_decay=weight_decay)    
    
# trainer = Trainer(cfg, model, optimizer, device=model._device)

In [6]:
torch.cuda.empty_cache()
it = 0
while True:
    it += 1
    print(it)
    data = next(iter(train_loader))
    print(torch.mean(data["actions"],(0,1)))
    out = model.forward(data)
    if torch.any(torch.isnan(out["mu"])):
        print(data["seq_pad_mask"])
        print(data["actions"][0,:,:])
        print(data["skill_pad_mask"])
        print(out["mu"])
        break
    del data
    del out
    torch.cuda.empty_cache()

1
tensor([-0.0007,  0.0120, -0.0021,  0.0029,  0.0031,  0.0144, -0.0037,  0.0675])
2
tensor([ 0.0054,  0.0120,  0.0003,  0.0017,  0.0021,  0.0157, -0.0080, -0.0362])
3
tensor([ 0.0100,  0.0124,  0.0012,  0.0076, -0.0014,  0.0061,  0.0054, -0.0262])
4
tensor([ 0.0358,  0.0148,  0.0174,  0.0465, -0.0121, -0.0202,  0.0023, -0.2212])
5
tensor([-4.8480e-03,  3.5061e-02,  5.7652e-04,  5.5036e-02, -9.0822e-05,
        -1.5334e-02, -1.3778e-02, -3.0000e-02])
6
tensor([-0.0047,  0.0004, -0.0057, -0.0253,  0.0043,  0.0304,  0.0035,  0.0213])
7
tensor([ 0.0210,  0.0037,  0.0020, -0.0057,  0.0002,  0.0143,  0.0103, -0.0712])
8
tensor([-0.0014,  0.0236,  0.0053,  0.0337, -0.0039, -0.0038, -0.0299,  0.0213])
9
tensor([ 0.0510,  0.0113,  0.0092,  0.0294, -0.0079, -0.0068,  0.0497, -0.2350])
10
tensor([ 0.0045, -0.0010,  0.0014,  0.0027, -0.0025,  0.0051, -0.0098, -0.0463])
11
tensor([ 0.0249,  0.0047,  0.0026,  0.0150, -0.0034, -0.0042,  0.0192, -0.1262])
12
tensor([ 0.0097,  0.0234,  0.0115,  0.0526

KeyboardInterrupt: 

In [4]:
data = next(iter(train_loader))
out = model.forward(data)